In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_column',None)
pd.set_option('display.max_row',1000)

from sklearn.model_selection import train_test_split

In [4]:
runs_raw = pd.read_csv('/home/goblin1732/Desktop/iMa_linux/runs.csv')
races = pd.read_csv('/home/goblin1732/Desktop/iMa_linux/races.csv')
runs_merge = runs_raw.merge(races, on='race_id')
runs_select = runs_merge[['race_id', 'horse_no', 'horse_id', 'result','horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'draw','finish_time',
       'win_odds', 'place_odds', 'trainer_id', 'jockey_id','race_no','distance','race_class']]

In [5]:
runs = runs_select.dropna()

In [6]:
count_horse = runs.groupby(['race_id']).horse_no.count()
count_horse['count_horse'] = runs.groupby(['race_id']).horse_no.count()
runs = runs.merge(count_horse, on='race_id')
#runs['group'] = runs.groupby(['horse_no_y']).ngroup()
runs['group'] = runs[['race_no']].copy()

In [7]:
runs['won'] = (runs['result'] ==1).astype(int)
runs['second'] = (runs['result'] ==2).astype(int)
runs['third'] = (runs['result'] ==3).astype(int)

runs['won_odds'] = runs.groupby('horse_id').won.cumsum()
runs['second_count'] = runs.groupby('horse_id').second.cumsum()
runs['third_count'] = runs.groupby('horse_id').third.cumsum()
runs['exp'] = runs.groupby('horse_id').horse_id.cumcount()
runs['raced']= (runs['exp'] > 0).astype(int)

runs['speed']= (runs['distance']/runs['finish_time']).round(4)
runs['last_speed'] = runs.groupby('horse_id').speed.shift(periods=1)
runs['fin_time'] = runs.groupby('horse_id').finish_time.shift(periods=1)
runs['prev_dist'] = runs.groupby('horse_id').distance.shift(periods=1)

runs['avg_2last']=((runs.groupby('horse_id')['result'].shift(periods=1) + runs.groupby('horse_id')['result'].shift(periods=2))/2)
runs.fillna(0,inplace=True)

In [8]:
shd = runs[['result','race_id','horse_no_y','horse_no_x','horse_id','jockey_id','trainer_id','horse_age','declared_weight',
            'actual_weight','draw','win_odds','place_odds','last_speed','avg_2last','won_odds',
            'second_count','third_count','exp','raced','fin_time','prev_dist','race_class',
            'horse_country', 'horse_type', 'horse_rating','horse_gear']]

In [9]:
runs

,race_id,horse_no_x,horse_id,result,horse_age,horse_country,horse_type,horse_rating,horse_gear,declared_weight,actual_weight,draw,finish_time,win_odds,place_odds,trainer_id,jockey_id,race_no,distance,race_class,horse_no_y,group,won,second,third,won_odds,second_count,third_count,exp,raced,speed,last_speed,fin_time,prev_dist,avg_2last
0,0,1,3917,10,3,AUS,Gelding,60,--,1020.0,133,7,83.92,9.7,3.7,118,2,1,1400,5,14,1,0,0,0,0,0,0,0,0,16.6826,0.0000,0.00,0.0,0.0
1,0,2,2157,8,3,NZ,Gelding,60,--,980.0,133,12,83.56,16.0,4.9,164,57,1,1400,5,14,1,0,0,0,0,0,0,0,0,16.7544,0.0000,0.00,0.0,0.0
2,0,3,858,7,3,NZ,Gelding,60,--,1082.0,132,8,83.40,3.5,1.5,137,18,1,1400,5,14,1,0,0,0,0,0,0,0,0,16.7866,0.0000,0.00,0.0,0.0
3,0,4,1853,9,3,SAF,Gelding,60,--,1118.0,127,13,83.62,39.0,11.0,80,59,1,1400,5,14,1,0,0,0,0,0,0,0,0,16.7424,0.0000,0.00,0.0,0.0
4,0,5,2796,6,3,GB,Gelding,60,--,972.0,131,14,83.24,50.0,14.0,9,154,1,1400,5,14,1,0,0,0,0,0,0,0,0,16.8188,0.0000,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75705,6128,10,1153,11,4,NZ,Gelding,65,--,1141.0,116,4,82.87,33.0,5.7,118,40,10,1400,3,14,10,0,0,0,2,1,2,8,1,16.8939,17.5593,56.95,1000.0,8.5
75706,6128,11,1925,6,6,GB,Gelding,64,H/XB,1066.0,110,6,82.42,99.0,14.0,73,161,10,1400,3,14,10,0,0,0,2,1,4,34,1,16.9862,17.0275,82.22,1400.0,8.5
75707,6128,12,2531,4,3,AUS,Gelding,63,B,1115.0,109,14,82.25,8.2,2.0,138,10,10,1400,3,14,10,0,0,0,1,2,1,6,1,17.0213,17.2075,81.36,1400.0,2.0
75708,6128,13,212,9,4,AUS,Horse,63,TT/H-/B,1180.0,117,7,82.66,59.0,9.9,7,23,10,1400,3,14,10,0,0,0,1,0,3,14,1,16.9368,16.8573,83.05,1400.0,12.0


In [10]:
shd_multindex = shd.set_index(['race_id','horse_no_x'])

In [11]:
shd_cleaned = shd_multindex.dropna()

In [12]:
shd_cleaned["jockey_id"].describe

<bound method NDFrame.describe of race_id  horse_no_x
0        1               2
         2              57
         3              18
         4              59
         5             154
                      ... 
6128     10             40
         11            161
         12             10
         13             23
         14            162
Name: jockey_id, Length: 75710, dtype: int64>

In [13]:
shd_test = shd_cleaned.copy()
shd_test['cum_sum_resu'] = shd_test.groupby(["horse_id"]).result.cumsum()
shd_test['cum_count_resu'] = shd_test.groupby(["horse_id"]).cumcount()
shd_test['cum_count_resu_shift'] = shd_test.groupby(["horse_id"]).cum_count_resu.shift(periods=-1)
shd_test['cum_avg_resu'] = shd_test['cum_sum_resu']/shd_test['cum_count_resu_shift']
shd_test['cum_avg_prev_resu'] = shd_test.groupby(["horse_id"]).cum_avg_resu.shift(periods=1)
shd_test['prev_resu'] = shd_test.groupby(["horse_id"]).result.shift(periods=1)
shd_test['prev_odds'] = shd_test.groupby(["horse_id"]).win_odds.shift(periods=1)
shd_test['prev_wt'] = shd_test.groupby(["horse_id"]).actual_weight.shift(periods=1)
shd_test['prev_declar_wt'] = shd_test.groupby(["horse_id"]).declared_weight.shift(periods=1)
#.shift(periods=-1)).where((shd_test.groupby(["horse_id"]).count()>=1),other=shd_test.groupby(["horse_id"]).result.shift(periods=0)))
#shd_test['expand_mean'] = shd_test.apply(((shd_test.groupby(["horse_id"]).result.cumsum() / shd_test.groupby(["horse_id"]).cumcount()).shift(periods=-1)).where((shd_test.groupby(["horse_id"]).count()>=2),other=shd_test.groupby(["horse_id"]).result.shift(periods=0)))

In [14]:
shd_clean = shd_test.drop(columns=['cum_count_resu_shift','cum_count_resu','cum_sum_resu','cum_avg_resu'])
shd_clean.columns

Index(['result', 'horse_no_y', 'horse_id', 'jockey_id', 'trainer_id',
       'horse_age', 'declared_weight', 'actual_weight', 'draw', 'win_odds',
       'place_odds', 'last_speed', 'avg_2last', 'won_odds', 'second_count',
       'third_count', 'exp', 'raced', 'fin_time', 'prev_dist', 'race_class',
       'horse_country', 'horse_type', 'horse_rating', 'horse_gear',
       'cum_avg_prev_resu', 'prev_resu', 'prev_odds', 'prev_wt',
       'prev_declar_wt'],
      dtype='object')

In [15]:
cols_nan =["last_speed","avg_2last","prev_dist","fin_time"]
shd_clean[cols_nan] = shd_clean[cols_nan].replace(0,np.nan)

In [16]:
shd_clean.horse_rating

race_id  horse_no_x
0        1             60
         2             60
         3             60
         4             60
         5             60
                       ..
6128     10            65
         11            64
         12            63
         13            63
         14            63
Name: horse_rating, Length: 75710, dtype: int64

In [25]:
shd_select = shd_clean[['result', 'horse_id', 'horse_no_y','horse_age', 'declared_weight',
       'actual_weight', 'draw', 'win_odds', 'place_odds','cum_avg_prev_resu', 'prev_resu']]

In [26]:
train_set1, test_set1 = train_test_split(shd_select, test_size=15144,shuffle=False)

all_train_set, all_test_set = train_test_split(shd_clean, test_size=15144,shuffle=False)

In [28]:
train_label1 = (train_set1["result"] <= 3.0).astype(int).copy()
test_label1 = (test_set1["result"] <= 3.0).astype(int).copy()
multi_train_label1 = train_set1["result"].copy()
multi_test_label1 = test_set1["result"].copy()
train_group1 = train_set1["horse_no_y"].copy()
test_group1 = test_set1["horse_no_y"].copy()
train_set1 = train_set1.drop(columns =['result'])
test_set1 = test_set1.drop(columns =['result'])

all_train_set = all_train_set.drop(columns =['result'])
all_test_set = all_test_set.drop(columns =['result'])

KeyError: 'result'

In [29]:
train_group1.to_csv('/home/goblin1732/Desktop/iMa_linux/train_group1.csv')
test_group1.to_csv('/home/goblin1732/Desktop/iMa_linux/test_group1.csv')

train_set1.to_csv('/home/goblin1732/Desktop/iMa_linux/train_set1.csv')
test_set1.to_csv('/home/goblin1732/Desktop/iMa_linux/test_set1.csv')

all_train_set.to_csv('/home/goblin1732/Desktop/iMa_linux/all_train_set.csv')
all_test_set.to_csv('/home/goblin1732/Desktop/iMa_linux/all_test_set.csv')

train_label1.to_csv('/home/goblin1732/Desktop/iMa_linux/train_label1.csv')
test_label1.to_csv('/home/goblin1732/Desktop/iMa_linux/test_label1.csv')

multi_train_label1.to_csv('/home/goblin1732/Desktop/iMa_linux/multi_train_label1.csv')
multi_test_label1.to_csv('/home/goblin1732/Desktop/iMa_linux/multi_test_label1.csv')

In [16]:
shd_select

result  group  horse_no_x  horse_age  declared_weight  \
race_id horse_id                                                          
0       3917          10      1           1          3           1020.0   
        2157           8      1           2          3            980.0   
        858            7      1           3          3           1082.0   
        1853           9      1           4          3           1118.0   
        2796           6      1           5          3            972.0   
...                  ...    ...         ...        ...              ...   
6128    1153          11     10          10          4           1141.0   
        1925           6     10          11          6           1066.0   
        2531           4     10          12          3           1115.0   
        212            9     10          13          4           1180.0   
        1229          10     10          14          3           1151.0   

                  actual_weight  draw  win_odds  place_odds  \
race_id horse_id                                              
0       3917                133     7       9.7         3.7   
        2157                133    12      16.0         4.9   
        858                 132     8       3.5         1.5   
        1853                127    13      39.0        11.0   
        2796                131    14      50.0        14.0   
...                         ...   ...       ...         ...   
6128    1153                116     4      33.0         5.7   
        1925                110     6      99.0        14.0   
        2531                109    14       8.2         2.0   
        212                 117     7      59.0         9.9   
        1229                116     9      99.0        26.0   

                  cum_avg_prev_resu  prev_resu  
race_id horse_id                                
0       3917                    NaN        NaN  
        2157                    NaN        NaN  
        858                     NaN        NaN  
        1853                    NaN        NaN  
        2796                    NaN        NaN  
...                             ...        ...  
6128    1153               4.250000        7.0  
        1925               7.029412       12.0  
        2531               4.500000        2.0  
        212                7.928571       14.0  
        1229               8.000000        8.0  

[75710 rows x 11 columns]

# imbalanced dataset label weights

In [51]:
shd_weights = runs.copy()
shd_weights['binary'] = (shd_weights["result"] <= 3.0).astype(int).copy()
n_samples = shd_weights.groupby('race_id').horse_no.count() 
n_classes = shd_weights.groupby('race_id').binary.sum()

AttributeError: 'DataFrameGroupBy' object has no attribute 'horse_no'

In [52]:
weight_samples = (n_samples / (n_samples - n_classes))
weight_classes = n_samples / (n_classes)
weights_data = {'n_samples': n_samples, 'count_classes': n_classes, 'weight_samples': weight_samples, 'weight_classes': weight_classes}
label_weights = pd.DataFrame(data=weights_data)

NameError: name 'n_samples' is not defined

In [126]:
label_weights_race = pd.merge(runs,label_weights,how='left', on='race_id')
label_weights_race.corrwith(label_weights_race['result'])

race_id              -0.000257
race_no               0.027185
surface              -0.004019
distance              0.001661
prize                -0.011319
race_class           -0.010945
sec_time1            -0.033984
sec_time2            -0.037086
sec_time3            -0.015424
sec_time4            -0.026768
sec_time5             0.002471
sec_time6             0.036272
sec_time7                  NaN
time1_x              -0.033984
time2_x              -0.035809
time3_x              -0.036093
time4_x              -0.058081
time5_x              -0.017665
time6_x              -0.041293
time7                      NaN
place_combination1    0.021603
place_combination2    0.023115
place_combination3    0.022223
place_combination4   -0.024839
place_dividend1       0.015787
place_dividend2       0.018518
place_dividend3       0.020062
place_dividend4       0.015135
win_combination1      0.021650
win_dividend1         0.010675
win_combination2     -0.062311
win_dividend2         0.036848
horse_no

#Groups Are Races with equal numbers of horses ?

In [127]:
runs.groupby('group').horse_no.value_counts()

KeyError: 'group'